In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from concurrent.futures import ProcessPoolExecutor
import time
import datetime
import io
import sqlite3
import pyodbc 
import concurrent
import threading


In [17]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(chrome_options=options)

<ipython-input-17-b5f1867c00b1>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [18]:
driver.get("https://www.google.com")

In [19]:
button=driver.find_element_by_xpath("//input")

In [20]:
button.send_keys("google play Instagram")

In [21]:
button.send_keys(Keys.RETURN)

In [22]:
#serach for the applcation in the Google play
try:
    link=WebDriverWait(driver,10).until(
        expected_conditions.presence_of_all_elements_located(
            (By.PARTIAL_LINK_TEXT, "Instagram – Apps on Google Play")
        )                
    )
    link[0].click()
except Exception as e:
    print(e)

In [23]:
def ful_reviws(i):
    result ='ful_reviws'
    try:
        
        ful_reviws=driver.find_element_by_xpath(
            F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span[1]/div/button'
            )
        ful_reviws.click()

    except :
        pass

def get_comments(i):
    result='get_comments'
     
    try:
        result= driver.find_elements_by_xpath(
                F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span'
                )
        result=''.join ([x.text for x in result ])
        
    except Exception as e:
        print(e) 
    return result

def get_author(i):
    result='get_author'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/span'
        ).text
    except Exception as e:
        print(e) 
    return result 

def get_rating(i):
    result='get_rating'
    try:
        rating=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[1]/div/div'
        ).get_attribute("aria-label")
        result =rating [6:7]
    except Exception as e:
        print(e) 
    return result 

def get_date(i):
    result='get_date'
    try:
        date=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[2]'
        ).text
        result=  datetime.datetime.strptime( date.replace(',',''), '%B %d %Y')
    except Exception as e:
        print(e) 
    return result 

def get_like(i):
    result='get_like'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div[1]/div[{i}]/div/div[2]/div[1]/div[2]/div/div[1]/div[2]'
        ).text
    except Exception as e:
        print(e) 
    return result 
             

In [24]:

class Multi_ins(threading.Thread):
    def insert(self,comment, author, rating,date, likes):
        connection = pyodbc.connect('Driver={SQL Server};'
                                    'Server='+'DESKTOP-RT88KQU'+';'
                                    'Database='+'socialMedia'+';'
                                    'user=sa;'
                                    'password=21')
        cursor = connection.cursor()
        cursor.execute("""
        INSERT INTO [dbo].[Instagram]([comment],[author],[rating],[date],[likes])VALUES (?,?,?,?,?)""",
        comment, author, rating, date, likes)
        connection.commit()

        def get_info(self, i):
            #try to pen comments
            ful_reviws(i)
            #get comments
            comment=get_comments(i)

            author=get_author(i)
            rating=get_rating(i)
            date=get_date(i)
            likes=get_like(i)

            # print(f"author:{author} rate: {rating} cr_date: {date} likes: {likes}")
            if (comment and comment !='get_comments' and comment !='') :
                self.insert(comment,author,rating ,date , likes  )

In [25]:
#show complete comments in the pfirst page 
#Todo : compete this part for the future use in the loop part 




In [26]:
#click on the read all revies for the first time in the page
try:
    read_all_reviews=WebDriverWait(driver, 4).until(
        expected_conditions.presence_of_element_located((By.XPATH, '//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span'))
    )  
except Exception as e:
    print(e)
read_all_reviews.click()
#//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span

In [27]:
def cuncur(j):
            with ProcessPoolExecutor() as executer:
                result = [executer.submit(get_info, j) for j in range(1, 10)]
                
            for item in concurrent.futures.as_completed(result):
                item.result()



In [29]:
def get_info( i):
    #try to pen comments
    ful_reviws(i)
    #get comments
    comment=get_comments(i)

    author=get_author(i)
    rating=get_rating(i)
    date=get_date(i)
    likes=get_like(i)

    # print(f"author:{author} rate: {rating} cr_date: {date} likes: {likes}")
    if (comment and comment !='get_comments' and comment !='') :
        Multi_ins().insert(comment,author,rating ,date , likes  )

In [28]:
j=1
flag=True
body = driver.find_element_by_css_selector('body')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)
while flag:
        for i in range(j, j+100):
            if j >4580:
                get_info(i)
        j+=100
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)            
        try:
            showmore=WebDriverWait(driver, 0.3).until(
                expected_conditions.presence_of_element_located(
                    (By.XPATH,'//div[@class="PFAhAf"]/div[@role="button"]/span[@class="CwaK9"]/span[@class="RveJvd snByac"]')
                )
            )
            showmore.click()
        except Exception as  e:
            pass
        if j % 50 == 0 :
            print(j)             
  

NameError: name 'get_info' is not defined